In [1]:
#Import Dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
#Load in data from csv
pipes_df = pd.read_csv("Pipe_data.csv")
pipes_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (57,59,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,FID,MOID,DIAMETER,PIPE_LENGTH,MATERIAL,DESCRIPTION,YEAR_INSTALLED,PRESSURE_ZONE,LINING,LINE_YEAR,...,WORK_ORDER_NUMBER,Comments,ReportDate,StartDateT,CompletedD,ReportedY,SrvcRqst_1,MATERIAL_1,MUSYM,FAILURE
0,16420,P35881,8.0,861.090481,M.J.,Distribution,1952,Hemp 1175,,0,...,1143689,PER UNIT 819\n\rREQUEST 1 6X12 PLATE AND 4 BAG...,2014-10-25,2014-11-16,2014-11-16,2014,Water Request,CAS,Ub,1
1,44465,P88079,8.0,749.511184,TYT,Distribution,1965,Hemp 1175,,0,...,1311148,MAIN BREAK,2015-08-18,2015-08-18,2015-08-18,2015,Water Request,DIP,ReD,1
2,23810,P53764,8.0,171.554040,M.J.,Distribution,1958,Hemp 1175,,0,...,1323748,PER JONATHAN WEBB THERE IS A MAIN BREAK AT THI...,2015-09-11,2015-09-12,2015-09-12,2015,Water Pressure Complaint,DIP,Ub,1
3,52630,P400741,8.0,42.761097,M.J.,Distribution,1958,Hemp 1175,,0,...,1323748,PER JONATHAN WEBB THERE IS A MAIN BREAK AT THI...,2015-09-11,2015-09-12,2015-09-12,2015,Water Pressure Complaint,DIP,Ub,1
4,10510,P20210,12.0,31.300481,DIP,Transmission,2006,Hemp 1175,,0,...,1628288,PER J. SKINNER MAIN BREAK AT LOCATION\nCall ta...,2017-01-22,2017-02-28,2017-02-28,2017,Turn off burst pipe,DIP,Ub,1


In [3]:
#clean data by converting all columns with strings to float/int or other type that works in ML 

#find all columns in dataframe where type = object 
df_types = pipes_df.select_dtypes(include=[np.object])

print(f'There are {len(df_types.dtypes)} columns containing strings')
print('--------------------------------------------------------------')
print(df_types.dtypes)

There are 23 columns containing strings
--------------------------------------------------------------
MOID                 object
MATERIAL             object
DESCRIPTION          object
PRESSURE_ZONE        object
LINING               object
PATTERN1             object
PATTERN2             object
PATTERN3             object
PATTERN4             object
PATTERN5             object
PATTERN6             object
PATTERN7             object
PATTERN8             object
PATTERN9             object
PATTERN10            object
WORK_ORDER_NUMBER    object
Comments             object
ReportDate           object
StartDateT           object
CompletedD           object
SrvcRqst_1           object
MATERIAL_1           object
MUSYM                object
dtype: object


In [4]:
#get dummies for columns where applicable--MATERIAL, DESCRIPTION, PRESSURE_ZONE
pipes_df = pd.concat([pipes_df, pd.get_dummies(pipes_df['MATERIAL'], prefix='MATERIAL')], axis=1)
pipes_df = pd.concat([pipes_df, pd.get_dummies(pipes_df['DESCRIPTION'], prefix='DESCRIPTION')], axis=1); 
pipes_df = pd.concat([pipes_df, pd.get_dummies(pipes_df['PRESSURE_ZONE'], prefix='P-ZONE')], axis=1); 
pipes_df.head()

,FID,MOID,DIAMETER,PIPE_LENGTH,MATERIAL,DESCRIPTION,YEAR_INSTALLED,PRESSURE_ZONE,LINING,LINE_YEAR,...,DESCRIPTION_Distribution,DESCRIPTION_Experimental Pipe not in CIP,DESCRIPTION_Facility Pipe,DESCRIPTION_Proposed Main: Adamsville,DESCRIPTION_Transmission,P-ZONE_Chatt 1020,P-ZONE_Hemp 1175,P-ZONE_Hemp 1175 (moved),P-ZONE_North 1225,P-ZONE_WTP
0,16420,P35881,8.0,861.090481,M.J.,Distribution,1952,Hemp 1175,,0,...,1,0,0,0,0,0,1,0,0,0
1,44465,P88079,8.0,749.511184,TYT,Distribution,1965,Hemp 1175,,0,...,1,0,0,0,0,0,1,0,0,0
2,23810,P53764,8.0,171.554040,M.J.,Distribution,1958,Hemp 1175,,0,...,1,0,0,0,0,0,1,0,0,0
3,52630,P400741,8.0,42.761097,M.J.,Distribution,1958,Hemp 1175,,0,...,1,0,0,0,0,0,1,0,0,0
4,10510,P20210,12.0,31.300481,DIP,Transmission,2006,Hemp 1175,,0,...,0,0,0,0,1,0,1,0,0,0


In [5]:
#drop original columns--MATERIAL, DESCRIPTION, PRESSURE_ZONE
pipes_df2 = pipes_df.drop(columns={ 'MATERIAL',
 'DESCRIPTION',
 'PRESSURE_ZONE',})
pipes_df2.head()

,FID,MOID,DIAMETER,PIPE_LENGTH,YEAR_INSTALLED,LINING,LINE_YEAR,INDEX_YR,ROUGHNESS,FLOW,...,DESCRIPTION_Distribution,DESCRIPTION_Experimental Pipe not in CIP,DESCRIPTION_Facility Pipe,DESCRIPTION_Proposed Main: Adamsville,DESCRIPTION_Transmission,P-ZONE_Chatt 1020,P-ZONE_Hemp 1175,P-ZONE_Hemp 1175 (moved),P-ZONE_North 1225,P-ZONE_WTP
0,16420,P35881,8.0,861.090481,1952,,0,1952,86,23.4711,...,1,0,0,0,0,0,1,0,0,0
1,44465,P88079,8.0,749.511184,1965,,0,1965,122,56.2002,...,1,0,0,0,0,0,1,0,0,0
2,23810,P53764,8.0,171.554040,1958,,0,1958,61,79.6832,...,1,0,0,0,0,0,1,0,0,0
3,52630,P400741,8.0,42.761097,1958,,0,1958,61,79.6832,...,1,0,0,0,0,0,1,0,0,0
4,10510,P20210,12.0,31.300481,2006,,0,2006,139,98.6106,...,0,0,0,0,1,0,1,0,0,0


In [7]:
# Now work on the'PATTERN' columns. There are 10 of them. 
#check if each pattern column only has 1 value
pipes_df2["PATTERN1"].value_counts()

RES    55937
          75
Name: PATTERN1, dtype: int64

In [8]:
pipes_df2["PATTERN2"].value_counts()

NON-RES    55937
              75
Name: PATTERN2, dtype: int64

In [9]:
pipes_df2["PATTERN3"].value_counts()

WHOLE    55937
            75
Name: PATTERN3, dtype: int64

In [10]:
pipes_df2["PATTERN4"].value_counts()

FLUSH    55937
            75
Name: PATTERN4, dtype: int64

In [11]:
pipes_df2["PATTERN5"].value_counts()

UFW    55937
          75
Name: PATTERN5, dtype: int64

In [12]:
pipes_df2["PATTERN6"].value_counts()

RES-DISTR    55937
                75
Name: PATTERN6, dtype: int64

In [13]:
pipes_df2["PATTERN7"].value_counts()

NON-RES-DISTR    55937
                    75
Name: PATTERN7, dtype: int64

In [14]:
pipes_df2["PATTERN8"].value_counts()

                56010
2030-CWTP-NS        2
Name: PATTERN8, dtype: int64

In [15]:
pipes_df2["PATTERN9"].value_counts()

     56012
Name: PATTERN9, dtype: int64

In [16]:
pipes_df2["PATTERN10"].value_counts()

     56012
Name: PATTERN10, dtype: int64

In [17]:
#Condense correlating PATTERN + DEMAND columns into 1 column
pipes_df2 = pipes_df2.rename(columns={"DEMAND1":"Demand_RES", "DEMAND2":"Demand_NON-RES","DEMAND3":"Demand_WHOLE","DEMAND4":"Demand_FLUSH","DEMAND5":"Demand_UFW",
                                     "DEMAND6":"Demand_RES-DISTR","DEMAND7":"Demand_NON RES-DISTR","DEMAND8":"Demand_2030-CWTP-NS"})

#Delete unneeded columns
#NOTE to Team: I didn’t see any data at all in the ‘DEMAND9’, ‘PATTERN9’, ‘DEMAND10’, ‘PATTERN10’ columns,so I deleted all of these columns
pipes_df3 = pipes_df2.drop(columns={"PATTERN1", "PATTERN2", "PATTERN3", "PATTERN4", "PATTERN5", "PATTERN6", "PATTERN7", "PATTERN8", "PATTERN9", "PATTERN10", "DEMAND9", "DEMAND10"})

In [18]:
pipes_df3.head()

,FID,MOID,DIAMETER,PIPE_LENGTH,YEAR_INSTALLED,LINING,LINE_YEAR,INDEX_YR,ROUGHNESS,FLOW,...,DESCRIPTION_Distribution,DESCRIPTION_Experimental Pipe not in CIP,DESCRIPTION_Facility Pipe,DESCRIPTION_Proposed Main: Adamsville,DESCRIPTION_Transmission,P-ZONE_Chatt 1020,P-ZONE_Hemp 1175,P-ZONE_Hemp 1175 (moved),P-ZONE_North 1225,P-ZONE_WTP
0,16420,P35881,8.0,861.090481,1952,,0,1952,86,23.4711,...,1,0,0,0,0,0,1,0,0,0
1,44465,P88079,8.0,749.511184,1965,,0,1965,122,56.2002,...,1,0,0,0,0,0,1,0,0,0
2,23810,P53764,8.0,171.554040,1958,,0,1958,61,79.6832,...,1,0,0,0,0,0,1,0,0,0
3,52630,P400741,8.0,42.761097,1958,,0,1958,61,79.6832,...,1,0,0,0,0,0,1,0,0,0
4,10510,P20210,12.0,31.300481,2006,,0,2006,139,98.6106,...,0,0,0,0,1,0,1,0,0,0


In [19]:
#ANUM'S STOPPING POINT
#These are the columns that still need to be converted to different types
df_types = pipes_df3.select_dtypes(include=[np.object])
df_types.dtypes

MOID                 object
LINING               object
WORK_ORDER_NUMBER    object
Comments             object
ReportDate           object
StartDateT           object
CompletedD           object
SrvcRqst_1           object
MATERIAL_1           object
MUSYM                object
dtype: object

In [20]:
#created new dataframe to change the datatype of ReportDate,StartDateT,CompletedD from object to datetime
df_date=pipes_df3

In [21]:
df_date["ReportDate"]=pd.to_datetime(df_date["ReportDate"], yearfirst=True, errors='coerce')
df_date["StartDateT"]=pd.to_datetime(df_date["StartDateT"],yearfirst=True, errors='coerce')
df_date["CompletedD"]=pd.to_datetime(df_date["CompletedD"],yearfirst=True, errors='coerce')

In [22]:
df_types11 = df_date.select_dtypes(include=[np.object])
df_types11.dtypes

MOID                 object
LINING               object
WORK_ORDER_NUMBER    object
Comments             object
SrvcRqst_1           object
MATERIAL_1           object
MUSYM                object
dtype: object

In [23]:
df_date["ReportDate"].head()

0   2014-10-25
1   2015-08-18
2   2015-09-11
3   2015-09-11
4   2017-01-22
Name: ReportDate, dtype: datetime64[ns]

In [24]:

df_date["CompletedD"].head()

0   2014-11-16
1   2015-08-18
2   2015-09-12
3   2015-09-12
4   2017-02-28
Name: CompletedD, dtype: datetime64[ns]

In [25]:
df_date["StartDateT"].head()

0   2014-11-16
1   2015-08-18
2   2015-09-12
3   2015-09-12
4   2017-02-28
Name: StartDateT, dtype: datetime64[ns]

In [26]:
pipes_df3.head()

,FID,MOID,DIAMETER,PIPE_LENGTH,YEAR_INSTALLED,LINING,LINE_YEAR,INDEX_YR,ROUGHNESS,FLOW,...,DESCRIPTION_Distribution,DESCRIPTION_Experimental Pipe not in CIP,DESCRIPTION_Facility Pipe,DESCRIPTION_Proposed Main: Adamsville,DESCRIPTION_Transmission,P-ZONE_Chatt 1020,P-ZONE_Hemp 1175,P-ZONE_Hemp 1175 (moved),P-ZONE_North 1225,P-ZONE_WTP
0,16420,P35881,8.0,861.090481,1952,,0,1952,86,23.4711,...,1,0,0,0,0,0,1,0,0,0
1,44465,P88079,8.0,749.511184,1965,,0,1965,122,56.2002,...,1,0,0,0,0,0,1,0,0,0
2,23810,P53764,8.0,171.554040,1958,,0,1958,61,79.6832,...,1,0,0,0,0,0,1,0,0,0
3,52630,P400741,8.0,42.761097,1958,,0,1958,61,79.6832,...,1,0,0,0,0,0,1,0,0,0
4,10510,P20210,12.0,31.300481,2006,,0,2006,139,98.6106,...,0,0,0,0,1,0,1,0,0,0


In [27]:
print([t for t in pipes_df3.dtypes])

[dtype('int64'), dtype('O'), dtype('float64'), dtype('float64'), dtype('int64'), dtype('O'), dtype('int64'), dtype('int64'), dtype('int64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('int64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('O'), dtype('O'), dtype('<M8[ns]'), dtype('<M8[ns]'), dtype('<M8[ns]'), dtype('int64'), dtype('O'), dtype('O'), dtype('O'), dtype('int64'), dtype('uint8'), dtype('uint8'), dtype('uint8'), dtype('uint8'), dtype('uint8'), dtype('uint8'), dtype('uint8'), dtype('ui

In [28]:
tempAgeList = []
for row in np.arange(len(pipes_df3['FID'])):
    try:
        value = int(pipes_df3.iloc[row]['ReportDate'].year) - pipes_df3.iloc[row]['YEAR_INSTALLED']
    except:
        value = 2018 - pipes_df3.iloc[row]['YEAR_INSTALLED']
    tempAgeList.append(value)

In [159]:
tempSuccessList = []
for row in np.arange(len(pipes_df3['FID'])):
    value = 1 - pipes_df3.iloc[row]['FAILURE']
    tempSuccessList.append(value)


KeyError: 'SUCCESS'

In [161]:
pipes_df3['SUCCESS'] = tempSuccessList

In [29]:
pipes_df3['Age']=tempAgeList

In [96]:
inputColumns = ['DIAMETER', 'PIPE_LENGTH','ROUGHNESS',
       'HL1000','AVE_FLOW', 'MAX_VELOCITY',
       'MIN_VELOCITY', 'AVE_VELOCITY',
       'AVE_HEADLOSS', 'ELEVATION','RUN_ELEV', 'DEMAND',
       'AVE_HEAD',
       'AVE_PRESS','MATERIAL_AMTYT',
       'MATERIAL_CI', 'MATERIAL_CO', 'MATERIAL_COP', 'MATERIAL_CPP',
       'MATERIAL_Copper', 'MATERIAL_DEL', 'MATERIAL_DIP', 'MATERIAL_EARGEO',
       'MATERIAL_GP', 'MATERIAL_HDPE', 'MATERIAL_L.J.', 'MATERIAL_M.J.',
       'MATERIAL_MCW', 'MATERIAL_OTH', 'MATERIAL_PE', 'MATERIAL_PP',
       'MATERIAL_PVC', 'MATERIAL_R.L.J.', 'MATERIAL_RCP', 'MATERIAL_SP',
       'MATERIAL_ST', 'MATERIAL_T.D', 'MATERIAL_T.D.', 'MATERIAL_TTE',
       'MATERIAL_TYT', 'MATERIAL_UNK', 'MATERIAL_W.I.','Age']

In [30]:
inputColumns = ['DIAMETER', 'PIPE_LENGTH','ROUGHNESS', 'FLOW', 'VELOCITY', 'HEADLOSS',
       'HL1000', 'MAX_FLOW', 'MIN_FLOW', 'AVE_FLOW', 'MAX_VELOCITY',
       'MIN_VELOCITY', 'AVE_VELOCITY', 'MAX_HEADLOSS', 'MIN_HEADLOSS',
       'AVE_HEADLOSS', 'ELEVATION','RUN_ELEV', 'DEMAND',
       'HEAD', 'PRESSURE', 'MAX_HEAD', 'MIN_HEAD', 'AVE_HEAD', 'MAX_PRESS',
       'MIN_PRESS', 'AVE_PRESS', 'DIFF_PRESS','MATERIAL_AMTYT',
       'MATERIAL_CI', 'MATERIAL_CO', 'MATERIAL_COP', 'MATERIAL_CPP',
       'MATERIAL_Copper', 'MATERIAL_DEL', 'MATERIAL_DIP', 'MATERIAL_EARGEO',
       'MATERIAL_GP', 'MATERIAL_HDPE', 'MATERIAL_L.J.', 'MATERIAL_M.J.',
       'MATERIAL_MCW', 'MATERIAL_OTH', 'MATERIAL_PE', 'MATERIAL_PP',
       'MATERIAL_PVC', 'MATERIAL_R.L.J.', 'MATERIAL_RCP', 'MATERIAL_SP',
       'MATERIAL_ST', 'MATERIAL_T.D', 'MATERIAL_T.D.', 'MATERIAL_TTE',
       'MATERIAL_TYT', 'MATERIAL_UNK', 'MATERIAL_W.I.','Age']

In [97]:
import random

In [98]:
X_train, X_test, y_train, y_test = train_test_split(pipes_df3[inputColumns],pipes_df3['FAILURE'],stratify=pipes_df3['FAILURE'],test_size = 0.3, random_state=42)

In [162]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(pipes_df3[inputColumns],pipes_df3[['FAILURE','SUCCESS']],stratify=pipes_df3['FAILURE'],test_size = 0.3, random_state=42)

In [99]:
model = LogisticRegression()

In [100]:
model.fit(X_train,y_train)
model.score(X_train,y_train)

0.9386349724546011

In [101]:
model.score(X_test,y_test)

0.9385860509402523

In [102]:
from sklearn.preprocessing import MinMaxScaler
X_minmax = MinMaxScaler().fit(X_train)
X_train_minmax = X_minmax.transform(X_train)
X_test_minmax = X_minmax.transform(X_test)


In [163]:
X_minmax_2 = MinMaxScaler().fit(X_train_2)
X_train_minmax_2 = X_minmax_2.transform(X_train_2)
X_test_minmax_2 = X_minmax_2.transform(X_test_2)

In [103]:
X_train_false = X_train[y_train==0].sample(n=sum(y_train))
X_train_true = X_train[y_train==1].sample(n=sum(y_train))

In [167]:
y_train_2

,FAILURE,SUCCESS
47519,0,1
52248,0,1
32504,0,1
54658,1,0
13889,0,1
35162,0,1
37998,0,1
44435,0,1
4200,1,0
54302,0,1


In [166]:
y_train

47519    0
52248    0
32504    0
54658    1
13889    0
35162    0
37998    0
44435    0
4200     1
54302    0
26136    0
46715    0
18542    0
26437    0
16475    0
6218     0
41463    0
16320    0
51353    0
23870    0
39950    0
33441    0
596      1
55885    0
24135    0
40793    0
51082    0
346      1
6453     0
20520    0
        ..
12444    0
44064    0
51335    0
1515     1
7247     0
5888     0
26523    0
8320     0
25132    0
34930    0
54496    0
29190    0
51500    0
53307    0
4929     0
8474     0
33728    0
43986    0
9269     0
425      1
52438    0
31946    0
25014    0
47646    0
18756    0
49879    0
4583     0
1210     0
22644    0
45095    0
Name: FAILURE, Length: 39208, dtype: int64

In [104]:
X_train_sampled = X_train_true.append(X_train_false)
y_train_sampled = y_train.loc[X_train_sampled.index]

In [173]:
y_train_sampled_2 = y_train_2.loc[X_train_sampled.index]

In [105]:
y_train_sampled.shape

(4776,)

In [106]:
X_train_minmax = X_minmax.transform(X_train_sampled)


## Logistic Regression

In [107]:
model = LogisticRegression()

In [108]:
model.fit(X_train_minmax,y_train_sampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [109]:
model.score(X_train_minmax,y_train_sampled)

0.7007956448911222

In [110]:
model.score(X_test_minmax,y_test)

0.7003094501309212

In [111]:
len(model.coef_[0])

43

In [112]:
len(inputColumns)

43

In [113]:
model.coef_[0]

array([-3.65505002e-01,  9.03048328e+00, -2.68452219e+00,  7.69496654e-02,
        1.48365769e+00,  5.16484085e-01,  1.84945409e+00,  7.80408085e-01,
        4.40705678e-01,  3.70068925e-01,  4.20160955e-01,  2.20239452e-01,
        1.02680561e+00,  1.62581890e+00,  0.00000000e+00, -3.38157169e-01,
        1.99429087e+00, -7.42075242e-01, -1.36754023e+00,  0.00000000e+00,
        3.79048381e-01, -1.83060214e-02,  0.00000000e+00, -4.38924034e-01,
        1.32560795e+00,  3.17198364e-03, -1.22056931e-01, -1.06993535e-01,
       -5.89870177e-01,  0.00000000e+00,  0.00000000e+00, -4.22653945e-01,
        0.00000000e+00,  1.25223051e+00, -6.63178447e-01,  4.02207831e-01,
        0.00000000e+00, -6.00510209e-01,  5.55242543e-01,  3.59501783e-03,
        1.18306488e-01, -1.75974602e-01, -3.31859963e+00])

In [114]:
from sklearn.metrics import classification_report
print(classification_report(y_test, model.predict(X_test_minmax)))

             precision    recall  f1-score   support

          0       0.97      0.70      0.81     15780
          1       0.13      0.68      0.22      1024

avg / total       0.92      0.70      0.78     16804



In [115]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, model.predict(X_test_minmax)))

[[11072  4708]
 [  328   696]]


In [116]:
zipped = zip(model.coef_[0],inputColumns)

In [117]:
list(zipped)

[(-0.3655050023905056, 'DIAMETER'),
 (9.030483282045898, 'PIPE_LENGTH'),
 (-2.6845221932270626, 'ROUGHNESS'),
 (0.07694966539797814, 'HL1000'),
 (1.4836576895247697, 'AVE_FLOW'),
 (0.5164840848085597, 'MAX_VELOCITY'),
 (1.8494540943760616, 'MIN_VELOCITY'),
 (0.7804080846840349, 'AVE_VELOCITY'),
 (0.4407056776943713, 'AVE_HEADLOSS'),
 (0.37006892497356675, 'ELEVATION'),
 (0.4201609548797858, 'RUN_ELEV'),
 (0.22023945165428968, 'DEMAND'),
 (1.0268056078005228, 'AVE_HEAD'),
 (1.6258188970861962, 'AVE_PRESS'),
 (0.0, 'MATERIAL_AMTYT'),
 (-0.3381571694063241, 'MATERIAL_CI'),
 (1.9942908728267783, 'MATERIAL_CO'),
 (-0.7420752422270527, 'MATERIAL_COP'),
 (-1.3675402299684567, 'MATERIAL_CPP'),
 (0.0, 'MATERIAL_Copper'),
 (0.37904838116615885, 'MATERIAL_DEL'),
 (-0.018306021416301787, 'MATERIAL_DIP'),
 (0.0, 'MATERIAL_EARGEO'),
 (-0.4389240344493731, 'MATERIAL_GP'),
 (1.3256079547494881, 'MATERIAL_HDPE'),
 (0.0031719836416887926, 'MATERIAL_L.J.'),
 (-0.1220569305339143, 'MATERIAL_M.J.'),
 (-0.1

## Random Forest

In [118]:
from sklearn.ensemble import RandomForestClassifier

In [178]:
rfmodel = RandomForestClassifier(n_estimators=300)

In [179]:
rfmodel.fit(X_train_minmax, y_train_sampled)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [180]:
rfmodel.score(X_train_minmax,y_train_sampled)

1.0

In [181]:
rfmodel.score(X_test_minmax,y_test)

0.8173649131159247

In [182]:
rfmodel.feature_importances_

array([4.46942057e-02, 1.03856655e-01, 6.25616504e-02, 4.72627154e-02,
       5.23351829e-02, 5.10167640e-02, 5.08232216e-02, 4.88865230e-02,
       5.96282306e-02, 5.09587275e-02, 5.11973101e-02, 6.90686811e-02,
       7.00685178e-02, 5.49542616e-02, 0.00000000e+00, 6.04721501e-04,
       6.79035501e-04, 6.36884297e-05, 7.55182340e-04, 0.00000000e+00,
       3.84730045e-03, 8.26205780e-03, 0.00000000e+00, 6.09959624e-05,
       4.25310720e-05, 3.71990684e-03, 1.07530995e-02, 8.44736370e-06,
       1.46549206e-04, 0.00000000e+00, 0.00000000e+00, 8.85811368e-05,
       0.00000000e+00, 1.37799121e-04, 2.10088790e-05, 4.27584211e-04,
       0.00000000e+00, 1.28954716e-02, 6.87979863e-06, 5.27386311e-03,
       1.91533788e-03, 1.22599503e-03, 1.31751316e-01])

In [183]:
zipped = zip(list(rfmodel.feature_importances_),inputColumns)


In [184]:
print(sorted(zip(map(lambda x: round(x, 4), rfmodel.feature_importances_), inputColumns), reverse=True))

[(0.1318, 'Age'), (0.1039, 'PIPE_LENGTH'), (0.0701, 'AVE_HEAD'), (0.0691, 'DEMAND'), (0.0626, 'ROUGHNESS'), (0.0596, 'AVE_HEADLOSS'), (0.055, 'AVE_PRESS'), (0.0523, 'AVE_FLOW'), (0.0512, 'RUN_ELEV'), (0.051, 'MAX_VELOCITY'), (0.051, 'ELEVATION'), (0.0508, 'MIN_VELOCITY'), (0.0489, 'AVE_VELOCITY'), (0.0473, 'HL1000'), (0.0447, 'DIAMETER'), (0.0129, 'MATERIAL_T.D.'), (0.0108, 'MATERIAL_M.J.'), (0.0083, 'MATERIAL_DIP'), (0.0053, 'MATERIAL_TYT'), (0.0038, 'MATERIAL_DEL'), (0.0037, 'MATERIAL_L.J.'), (0.0019, 'MATERIAL_UNK'), (0.0012, 'MATERIAL_W.I.'), (0.0008, 'MATERIAL_CPP'), (0.0007, 'MATERIAL_CO'), (0.0006, 'MATERIAL_CI'), (0.0004, 'MATERIAL_ST'), (0.0001, 'MATERIAL_RCP'), (0.0001, 'MATERIAL_PVC'), (0.0001, 'MATERIAL_OTH'), (0.0001, 'MATERIAL_GP'), (0.0001, 'MATERIAL_COP'), (0.0, 'MATERIAL_TTE'), (0.0, 'MATERIAL_T.D'), (0.0, 'MATERIAL_SP'), (0.0, 'MATERIAL_R.L.J.'), (0.0, 'MATERIAL_PP'), (0.0, 'MATERIAL_PE'), (0.0, 'MATERIAL_MCW'), (0.0, 'MATERIAL_HDPE'), (0.0, 'MATERIAL_EARGEO'), (0.0, 

In [185]:
inputColumns

['DIAMETER',
 'PIPE_LENGTH',
 'ROUGHNESS',
 'HL1000',
 'AVE_FLOW',
 'MAX_VELOCITY',
 'MIN_VELOCITY',
 'AVE_VELOCITY',
 'AVE_HEADLOSS',
 'ELEVATION',
 'RUN_ELEV',
 'DEMAND',
 'AVE_HEAD',
 'AVE_PRESS',
 'MATERIAL_AMTYT',
 'MATERIAL_CI',
 'MATERIAL_CO',
 'MATERIAL_COP',
 'MATERIAL_CPP',
 'MATERIAL_Copper',
 'MATERIAL_DEL',
 'MATERIAL_DIP',
 'MATERIAL_EARGEO',
 'MATERIAL_GP',
 'MATERIAL_HDPE',
 'MATERIAL_L.J.',
 'MATERIAL_M.J.',
 'MATERIAL_MCW',
 'MATERIAL_OTH',
 'MATERIAL_PE',
 'MATERIAL_PP',
 'MATERIAL_PVC',
 'MATERIAL_R.L.J.',
 'MATERIAL_RCP',
 'MATERIAL_SP',
 'MATERIAL_ST',
 'MATERIAL_T.D',
 'MATERIAL_T.D.',
 'MATERIAL_TTE',
 'MATERIAL_TYT',
 'MATERIAL_UNK',
 'MATERIAL_W.I.',
 'Age']

In [186]:
sorted([1,3,5,2,4])

[1, 2, 3, 4, 5]

In [187]:
print(confusion_matrix(y_test, rfmodel.predict(X_test_minmax)))

[[12922  2858]
 [  211   813]]


## Gridsearching

In [128]:
model = RandomForestClassifier()
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators': [20,50,100,200]}
grid = GridSearchCV(model, param_grid, verbose=3)


In [129]:
grid.fit(X_train_minmax,y_train_sampled)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] n_estimators=20 .................................................
[CV] ........ n_estimators=20, score=0.7952261306532663, total=   0.1s
[CV] n_estimators=20 .................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV] ........ n_estimators=20, score=0.7713567839195979, total=   0.2s
[CV] n_estimators=20 .................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s


[CV] ......... n_estimators=20, score=0.792713567839196, total=   0.1s
[CV] n_estimators=50 .................................................
[CV] ........ n_estimators=50, score=0.8046482412060302, total=   0.5s
[CV] n_estimators=50 .................................................
[CV] ........ n_estimators=50, score=0.7870603015075377, total=   0.5s
[CV] n_estimators=50 .................................................
[CV] ........ n_estimators=50, score=0.8008793969849246, total=   0.5s
[CV] n_estimators=100 ................................................
[CV] ....... n_estimators=100, score=0.8027638190954773, total=   1.3s
[CV] n_estimators=100 ................................................
[CV] ....... n_estimators=100, score=0.7933417085427136, total=   1.1s
[CV] n_estimators=100 ................................................
[CV] ....... n_estimators=100, score=0.8090452261306532, total=   1.0s
[CV] n_estimators=200 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:   14.1s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [20, 50, 100, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [130]:
grid.score(X_test_minmax,y_test)

0.818079028802666

In [131]:
grid.best_params_

{'n_estimators': 200}

In [134]:
print(classification_report(y_test, grid.predict(X_test_minmax)))

             precision    recall  f1-score   support

          0       0.98      0.82      0.89     15780
          1       0.22      0.79      0.35      1024

avg / total       0.94      0.82      0.86     16804



## Neural Network

In [144]:
import tensorflow as tf
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [147]:
nnmodel = Sequential()

In [148]:
nnmodel.add(Dense(100, activation='relu', input_dim=X_train_minmax.shape[1]))

In [169]:
nnmodel.add(Dense(2,activation = 'softmax'))

In [170]:
nnmodel.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])

In [174]:
nnmodel.fit(
    X_train_minmax,
    y_train_sampled_2,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
 - 1s - loss: 0.6623 - acc: 0.6097
Epoch 2/100
 - 0s - loss: 0.6392 - acc: 0.6336
Epoch 3/100
 - 0s - loss: 0.6279 - acc: 0.6434
Epoch 4/100
 - 0s - loss: 0.6163 - acc: 0.6642
Epoch 5/100
 - 0s - loss: 0.6043 - acc: 0.6765
Epoch 6/100
 - 0s - loss: 0.5910 - acc: 0.6972
Epoch 7/100
 - 0s - loss: 0.5851 - acc: 0.7014
Epoch 8/100
 - 0s - loss: 0.5751 - acc: 0.7155
Epoch 9/100
 - 0s - loss: 0.5701 - acc: 0.7178
Epoch 10/100
 - 0s - loss: 0.5621 - acc: 0.7289
Epoch 11/100
 - 0s - loss: 0.5558 - acc: 0.7318
Epoch 12/100
 - 0s - loss: 0.5509 - acc: 0.7324
Epoch 13/100
 - 0s - loss: 0.5471 - acc: 0.7379
Epoch 14/100
 - 0s - loss: 0.5444 - acc: 0.7397
Epoch 15/100
 - 0s - loss: 0.5387 - acc: 0.7458
Epoch 16/100
 - 0s - loss: 0.5364 - acc: 0.7425
Epoch 17/100
 - 0s - loss: 0.5299 - acc: 0.7571
Epoch 18/100
 - 0s - loss: 0.5270 - acc: 0.7485
Epoch 19/100
 - 0s - loss: 0.5219 - acc: 0.7563
Epoch 20/100
 - 0s - loss: 0.5167 - acc: 0.7592
Epoch 21/100
 - 0s - loss: 0.5169 - acc: 0.7609
E

In [177]:
nnmodel_loss, nnmodel_accuracy = nnmodel.evaluate(X_test_minmax, y_test_2, verbose=2)
print(f"Loss: {nnmodel_loss}, Accuracy: {nnmodel_accuracy}")

Loss: 0.5590688051197875, Accuracy: 0.748690787907641


## Finding those pipes about to break (but haven't yet)

In [189]:
rfmodel.predict_proba(X_test_minmax)

array([[0.61666667, 0.38333333],
       [0.84666667, 0.15333333],
       [0.79      , 0.21      ],
       ...,
       [0.86333333, 0.13666667],
       [0.90666667, 0.09333333],
       [0.90333333, 0.09666667]])

In [213]:
X_test_output = X_test
X_test_output['BurstChance'] = [x[1] for x in list(rfmodel.predict_proba(X_test_minmax))]

In [214]:
X_test_output['FAILURE'] = y_test

In [221]:
X_test_output.loc[X_test_output['FAILURE']==0].sort_values('BurstChance',ascending=False).head(10)

,DIAMETER,PIPE_LENGTH,ROUGHNESS,HL1000,AVE_FLOW,MAX_VELOCITY,MIN_VELOCITY,AVE_VELOCITY,AVE_HEADLOSS,ELEVATION,...,MATERIAL_ST,MATERIAL_T.D,MATERIAL_T.D.,MATERIAL_TTE,MATERIAL_TYT,MATERIAL_UNK,MATERIAL_W.I.,Age,BurstChance,FAILURE
30893,6.0,563.533823,84,0.0026,2.6551,0.0335,0.0240,0.0301,0.0016,831.155278,...,0,0,0,0,0,0,0,59,0.983333,0
29863,6.0,656.275538,84,0.0020,2.4408,0.0320,0.0198,0.0277,0.0016,856.000000,...,0,0,0,0,0,0,0,60,0.970000,0
29050,6.0,583.595496,84,0.0013,1.8730,0.0251,0.0143,0.0213,0.0009,847.000000,...,0,0,0,0,0,0,0,61,0.966667,0
30862,6.0,459.069308,84,0.0048,6.3433,0.1677,0.0015,0.0720,0.0089,873.813172,...,0,0,0,0,0,0,0,59,0.963333,0
28837,6.0,594.058805,84,0.0010,1.6503,0.0210,0.0147,0.0187,0.0007,826.321512,...,0,0,0,0,0,0,0,61,0.960000,0
8266,36.0,104.262509,152,0.1065,4103.7539,1.7175,0.6711,1.2935,0.0132,917.255903,...,0,0,0,0,0,0,0,20,0.960000,0
28050,6.0,501.267553,84,0.0205,7.6829,0.1592,0.0201,0.0872,0.0115,816.984573,...,0,0,0,0,0,0,0,62,0.956667,0
28920,6.0,498.366119,84,0.0044,3.6030,0.0476,0.0287,0.0409,0.0025,849.331157,...,0,0,0,0,0,0,0,61,0.956667,0
28881,6.0,672.217027,84,0.0044,3.5178,0.0450,0.0307,0.0399,0.0032,875.741025,...,0,0,0,0,0,0,0,61,0.956667,0
32756,6.0,538.801083,84,0.4094,42.6443,0.6626,0.2538,0.4839,0.2703,927.565163,...,0,0,0,0,0,0,0,52,0.950000,0


In [219]:
pipes_df3.iloc[30893]

FID                                             8589
MOID                                          P28339
DIAMETER                                           6
PIPE_LENGTH                                  563.534
YEAR_INSTALLED                                  1959
LINING                                              
LINE_YEAR                                          0
INDEX_YR                                        1959
ROUGHNESS                                         84
FLOW                                          2.5308
VELOCITY                                      0.0287
HEADLOSS                                      0.0015
HL1000                                        0.0026
MAX_FLOW                                      2.9549
MIN_FLOW                                      2.1134
AVE_FLOW                                      2.6551
MAX_VELOCITY                                  0.0335
MIN_VELOCITY                                   0.024
AVE_VELOCITY                                  

## Cross Validation

In [224]:
from sklearn.model_selection import StratifiedKFold

In [225]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [226]:
split = cv.split(pipes_df3[inputColumns],pipes_df3['FAILURE'])

TypeError: 'generator' object is not subscriptable

In [239]:
scores = []
rfmodel = RandomForestClassifier(n_estimators=300)
final_df = pipes_df3
final_df['BurstChance'] = 0

In [240]:
final_df['BurstChance']

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
55982    0
55983    0
55984    0
55985    0
55986    0
55987    0
55988    0
55989    0
55990    0
55991    0
55992    0
55993    0
55994    0
55995    0
55996    0
55997    0
55998    0
55999    0
56000    0
56001    0
56002    0
56003    0
56004    0
56005    0
56006    0
56007    0
56008    0
56009    0
56010    0
56011    0
Name: BurstChance, Length: 56012, dtype: int64

In [247]:


for train_idx, test_idx in cv.split(pipes_df3[inputColumns],pipes_df3['FAILURE']):
    Current_X_train = pipes_df3[inputColumns].loc[train_idx]
    Current_X_test = pipes_df3[inputColumns].loc[test_idx]
    Current_y_train = pipes_df3['FAILURE'].loc[train_idx]
    Current_y_test = pipes_df3['FAILURE'].loc[test_idx]
    
    Current_X_minmax = MinMaxScaler().fit(Current_X_train)
    Current_X_train_minmax = Current_X_minmax.transform(Current_X_train)
    Current_X_test_minmax = Current_X_minmax.transform(Current_X_test)
    
    Current_X_train_false = Current_X_train[Current_y_train==0].sample(n=sum(Current_y_train))
    Current_X_train_true = Current_X_train[Current_y_train==1].sample(n=sum(Current_y_train))
    
    Current_X_train_sampled = Current_X_train_true.append(Current_X_train_false)
    Current_y_train_sampled = Current_y_train.loc[Current_X_train_sampled.index]
    
    Current_X_train_minmax = Current_X_minmax.transform(Current_X_train_sampled)

    
    rfmodel.fit(Current_X_train_minmax, Current_y_train_sampled)
    score = rfmodel.score(Current_X_test_minmax, Current_y_test)
    scores.append(score)
    print(confusion_matrix(Current_y_test,rfmodel.predict(Current_X_test_minmax)))
    
    final_df['BurstChance'].loc[test_idx] = [x[1] for x in list(rfmodel.predict_proba(Current_X_test_minmax))]
    
scores

[[8642 1878]
 [ 111  572]]


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


[[8761 1759]
 [ 128  555]]


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


[[8812 1708]
 [ 117  565]]


...

[[8568 1952]
 [ 125  557]]


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


[[8694 1826]
 [ 139  543]]


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


[0.8162992055699366,
 0.8372757297152549,
 0.8269951794322442,
 0.8222638814497412,
 0.8221746116764863,
 0.8192448451307686,
 0.8288851200571276,
 0.829851812176397,
 0.8201214068916265,
 0.8194965184788431,
 0.8224582701062215,
 0.8315629742033384,
 0.8370826638100339,
 0.8145866809498303,
 0.8245848955543653]

In [252]:
final_df[['BurstChance','FID','MOID'] + inputColumns].loc[final_df['FAILURE']==0].sort_values('BurstChance', ascending=False).head(10)

,BurstChance,FID,MOID,DIAMETER,PIPE_LENGTH,ROUGHNESS,HL1000,AVE_FLOW,MAX_VELOCITY,MIN_VELOCITY,...,MATERIAL_RCP,MATERIAL_SP,MATERIAL_ST,MATERIAL_T.D,MATERIAL_T.D.,MATERIAL_TTE,MATERIAL_TYT,MATERIAL_UNK,MATERIAL_W.I.,Age
2008,0.973333,16852,P38683,6.0,523.548036,39,1.1411,44.6551,0.6175,0.3268,...,0,0,0,0,0,0,0,0,0,80
30893,0.973333,8589,P28339,6.0,563.533823,84,0.0026,2.6551,0.0335,0.0240,...,0,0,0,0,0,0,0,0,0,59
2313,0.966667,16387,P35796,6.0,690.583405,39,0.0843,12.2440,0.1814,0.0440,...,0,0,0,0,0,0,0,0,0,75
18807,0.960000,153,P106854,6.0,1283.013408,39,0.2037,10.1875,0.2039,0.0279,...,0,0,0,0,0,0,0,0,0,67
2119,0.960000,912,P103019,6.0,636.358576,39,0.0631,15.3874,0.2520,0.0252,...,0,0,0,0,0,0,0,0,0,78
55769,0.953333,53927,P45877,2.0,328.264650,34,0.3194,0.8229,0.0984,0.0580,...,0,0,0,0,0,0,0,0,1,56
55684,0.953333,54071,P47905,2.0,415.751783,34,0.3990,0.9316,0.1122,0.0641,...,0,0,0,0,0,0,0,0,1,59
55878,0.950000,53929,P46994,2.0,311.496146,40,0.3068,0.9513,0.1148,0.0652,...,0,0,0,0,0,0,0,0,1,52
32369,0.950000,8001,P30545,6.0,702.251530,90,0.2293,28.1961,0.5759,0.0279,...,0,0,0,0,0,0,0,0,0,55
30842,0.950000,5084,P18742,6.0,704.887925,84,0.1998,32.9333,0.4761,0.2464,...,0,0,0,0,0,0,0,0,0,59
